# FIT5196 Task 2 in Assessment 1
#### Student Name: Anil Gurbuz
#### Student ID: 29628792

Date: XX/XX/XXXX

Version: 1.0

Environment: Python 2.7.11 and Jupyter notebook

Libraries used: 
* pandas ( included in Anaconda Python 2.7) 
* re (for regular expression, included in Anaconda Python 2.7)
* nltk.stem.porter
* sklearn.feature_extraction.tex
 



 ## Reading Resumes

In [107]:
import re
import pandas as pd
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer

resumeno=[  457, 119, 612 , 391, 695, 245, 730, 607, 474, 338, 382, 10, 481, 339, 584, 820, 722, 610,
 858,  13, 237, 109, 537, 523, 331, 429, 293, 277, 437, 459, 432, 452, 197, 522, 317, 706,
  44, 770, 388, 493,  77, 503, 218, 774, 363, 126, 687, 666, 771, 672,  41, 711, 684, 614,
 123, 232, 590,  38, 523,  15, 566, 444, 559,   8, 377, 692, 591, 403, 268,  63, 552, 859,
 333, 645, 468, 372, 143, 853, 146, 669,  76, 250, 457, 704, 761, 803, 244, 751, 472, 293,
  47, 389, 278, 610, 679, 416, 165, 106, 479, 763, 261, 739, 746, 494, 356, 522, 100, 673,
 825,  22, 568, 302, 598, 459, 130, 699, 504, 630, 509, 378, 167, 740, 742, 472, 550, 834,
 384, 772, 242, 431, 845, 414, 640, 227, 488, 821, 188, 661, 838, 705,  58, 207, 493,   8,
 528, 571, 824, 584, 721, 192, 118, 241, 113, 712, 568, 178, 323, 309, 118, 576, 312, 175,
 272, 398,  42, 634, 608, 860, 835, 859, 325, 818,  17, 640, 799, 726, 624, 160, 650, 759,
 360, 115,  17, 428,  64, 542, 210, 688, 454, 194, 788, 789, 247, 397, 464, 782, 609, 508,
  33, 611, 571,  97, 514, 286, 350, 648, 482,  35, 394, 203, 665, 733, 285, 354, 344,  59,
 215, 483, 450, 707, 454, 282, 107,  96, 829, 594, 429, 646,  69, 273, 635, 323, 160, 308,
 508, 522, 493, 586, 860, 300, 465, 458, 774,  85, 849, 162, 688, 144, 503, 519]

Resumes are read by using regex and resumes list created above to specify the name of the resume files.

In [108]:
list_resumes=[]


for each in resumeno:
    data = open(r"resumeTxt/resume_(%s).txt" %each)
    list_resumes.append(data.read())
    

Getting the stopwords and structuring it to use in the upcoming steps.

In [109]:
stopwords=open('stopwords_en.txt')
stopwords=stopwords.readlines()

real_stopwords=[]

for i in range(len(stopwords)):
    stopwords[i]=stopwords[i].replace('\n','')
    
    if len(stopwords[i]) > 1:
        real_stopwords.append(stopwords[i])  

## Tokenization ##

In [110]:
len(list_resumes)

250

list_resumes is a list that has 250 entries and every entry is a string containing an entire resume. 

At this step, i will tokenize the resumes using the regex , specified at task2 part of the assignment.
Then i create a list of list ( resumes) which includes 250 lists inside ( like list_resumes variable) and every list has tokens of its resume.

In [111]:
resumes=[]

for resume in list_resumes:
    tokens = re.findall(r"\w+(?:[-']\w+)?", resume)
    resumes.append(tokens)


## Stemming - Stop words removal - digit removal - less than 2 char-token removal

At this step, 

* To remove the tokens containing digits, a simple regex used to find the digit in every token and if there is a digit in the token, removed.

* For stemming, each resume is taken using for loop from resumes variable than every token in the each resume is taken again with for loop.
        * First these tokens are stemmed using PorterStemmer.
        * Then length of these stemmed tokens are checked and if it is less than 3, removed.
        * Then stemmed tokens checked again to make sure they are not in the stop words list.
        * Lastly again each token checked whether they include a digit or not, if so removed.
* Then a temp list is created for every resume and if a token passes from every criterias defined above, added to that list to be added to another list in loop

* Last step is to add these temporary lists to a new list which is named as "token_stemmed". This list again contains lists whose content is the all appropriate tokens for a resume

In [112]:
token_stemmed=[]
no_digits = re.compile(r'\d+', re.M)

for resume in resumes:
    temp=[]
    
    for token in resume:
        stemmed_word = PorterStemmer().stem(token)
        if (len(stemmed_word)> 2) and (stemmed_word not in real_stopwords):
            if not re.match(no_digits, stemmed_word):
                temp.append(stemmed_word)
    
    token_stemmed.append(temp) 

In order to prepare our data structure for the CountVectorizer function, i created a list of string. Every string's content is the all appropriate tokens of corresponding resume of string.

In [113]:
resumes = []
for doc in token_stemmed:
    resume = ' '.join(doc)
    resumes.append(resume)
    
print(len(resumes))

250


## Vectorization

In this part of the assignment, rare tokens and the tokens has the frequency higher than 98% is removed using CountVectorizer fuinction and 200 most meaningfull bigrams are generated.

In [114]:
oneword = CountVectorizer(analyzer='word',
                        ngram_range=(1,1),
                        # tokens must appear in more than 2 percent of the documents
                       min_df=0.02,
                        # tokens
                       max_df=0.98)

twoword = CountVectorizer(analyzer='word',
                # tokens must appear in more than 2 percent of the documents
               min_df=0.02,
                # tokens
               max_df=0.98,
               ngram_range=(2,2),
                max_features=200,
                tokenizer=None,
                preprocessor=None)



 ## Output Files

The code below generates the sparse representation of the resumes and creates the output with specified format.
Ordered in alphabetic order. 

In [115]:
sparse1 = oneword.fit_transform(resumes)
sparse2 = twoword.fit_transform(resumes)

unigrams = oneword.get_feature_names()
bigrams = twoword.get_feature_names()
vectors = unigrams + bigrams




vectors.sort()

output1=''

for index, vector in enumerate(vectors):
    line = f'%s : %s' %(index, vector)
    output1 += line + '\n'
    
with open('29628792_vocab.txt', 'w') as vocab:
    vocab.write(output1)

First chunk of code, creates two dataframes having content of unigrams-bigrams(respectively) and their frequencies in every resume.

Then vector_rep variable is the combined dataframe of these 2 dataframes whose columns are sorted alphabeticly.

In [116]:
unigram_df = pd.DataFrame(sparse1.todense(), columns=unigrams)
# bigram_df =
bigram_df = pd.DataFrame(sparse2.todense(), columns=bigrams)

vector_rep = pd.concat([unigram_df, bigram_df], axis=1).sort_index(axis=1)

    

The code below produces the output in the specified format. First uses a for loop to reach the which resume is considered then prints the resume number using the firstly created resumeno list variable. Then another for loop takes every column name to look at every values for every resume's every token. If the value is not zero, adds the vector index ( from vectors variable) and the corresponding frequency from(vector_rep variable). At the end writes the whole string in the file.

In [117]:
string=''
for resume_no in range(len(vector_rep.index)) :
    string+='resume_(%s).txt, ' %resumeno[resume_no]
    for colmn in vector_rep.columns:
        if vector_rep[colmn][resume_no] !=0:
            string+= str(vectors.index(colmn)) + ':' + str(vector_rep[colmn][resume_no]) + ', '
    string = string[:-2]
    string+='\n'


    
with open('29628792_countVec.txt','w') as countvec:
    countvec.write(string)
                
    